In [1]:
import pickle

tweets = []
tmin = 500
tmax = 479500 # latest file name

for i in range(tmin, tmax + 1, 500):
    file_name = "../dataKA/all_tweets/tweets" + str(i)
    file_object = open(file_name, 'rb')
    for tweet in pickle.load(file_object):
        tweets.append(tweet)

In [2]:
keywords = ["covid", "covid19", "covid2019", "covid-19", "covid-2019", "covid_19", "covid_2019", "COVIDー19",
            "COVIDー2019", "corona", "virus", "coronavirus", "coronakrise", "coronacrisis", "corona-crise", 
            "pandemic", "coronapandemic", "coronavirusapandemic", "covidpandemic", "covid19pandemic", "covid2019pandemic" 
            "iorestoacasa", "forzalombardia", "quarantena", "covidswitzerland", "wtfockdown", "stayhome", "stayathome",
            "staysafe", "stayhomesavelives", "stayhomestaysafe", "lockdown", "socialdistancing", "distancing", "quarantine",
            "quarantinelife", "confinement", "confinementjour", "restezchezvous", "lavuedepuismonconfinement", 
            "BloqueoNoSolidaridadSi", "protectyourselfandothers", "coronavirusdelcastigominore", "coronainfoch", "masken",
            "masques", "masks", "homeoffice", "wfh", "workfromhome", "BAG_OFSP_UFSP", "restriktionen"] 


# from Karolos
def check(words, sentence):
    for word in words:
        if word in sentence:
            return True
    return False

keywords_lower = [k.lower() for k in keywords]

In [4]:
# COVID-related
it_cov_date = []
it_cov_tid = []
it_cov_text = []
it_cov_tweets = []

cnt = 0

for tweet in tweets:
    
    if check(keywords_lower, tweet["text"].lower()):
        if tweet["lang"] == "it":
            it_cov_tweets.append(tweet)
            it_cov_tid.append(tweet["id"])
            it_cov_date.append(tweet["created_at"])
            try:
                it_cov_text.append(tweet["extended_tweet"]["full_text"])
            except KeyError:
                it_cov_text.append(tweet["text"])
            
    elif "extended_tweet" in tweet and check(keywords_lower, tweet["extended_tweet"]["full_text"]):
        if tweet["lang"] == "it":
            cnt = cnt + 1
            it_cov_tweets.append(tweet)
            it_cov_tid.append(tweet["id"])
            it_cov_date.append(tweet["created_at"])
            it_cov_text.append(tweet["extended_tweet"]["full_text"])
            
    elif "retweeted_status" in tweet and check(keywords_lower, tweet["retweeted_status"]["full_text"]):
        if tweet["lang"] == "it":
            cnt = cnt + 1
            it_cov_tweets.append(tweet)
            it_cov_tid.append(tweet["id"])
            it_cov_date.append(tweet["created_at"])
            it_cov_text.append(tweet["retweeted_status"]["full_text"])
            
    elif "quoted_status" in tweet and check(keywords_lower, tweet["quoted_status"]["text"]):
        if tweet["lang"] == "it":
            cnt = cnt + 1
            it_cov_tweets.append(tweet)
            it_cov_tid.append(tweet["id"])
            it_cov_date.append(tweet["created_at"])
            it_cov_text.append(tweet["quoted_status"]["text"])

print("No. IT covid tweets: ", cnt)

No. IT covid tweets:  736


In [6]:
# All IT tweets
it_all_date = []
it_all_tid = []
it_all_text = []
it_all_tweets = []

for tweet in tweets:
    if tweet['lang'] == 'it':
        it_all_tweets.append(tweet)
        it_all_tid.append(tweet["id"])
        it_all_date.append(tweet["created_at"])
        try:
            it_all_text.append(tweet["extended_tweet"]["full_text"])
        except KeyError:
            it_all_text.append(tweet["text"])

print("No. IT tweets (total): ", len(it_all_tid))

No. IT tweets (total):  43481


In [7]:
import pandas as pd
import numpy as np


## Export full tweet datasets ##

it_all = pd.concat([pd.Series(it_all_tid), 
                           pd.Series(it_all_date), 
                           pd.Series(it_all_text)],
                          axis = 1)
it_covid = pd.concat([pd.Series(it_cov_tid),
                             pd.Series(it_cov_date),
                             pd.Series(it_cov_text)],
                            axis = 1)

# it_all.to_csv("../dataKA/latest/it_tweets_all.csv", encoding = "utf-8")
# it_covid.to_csv("../dataKA/latest/it_tweets_covid.csv", encoding = "utf-8")

## Export text of covid dataset and full dataset for tool training ## 
out_it_all_text = pd.DataFrame(it_all_text)
out_it_cov_text = pd.DataFrame(it_cov_text)

# out_it_all_text.to_csv("../dataKA/latest/it_alltext.csv", encoding = "utf-8")
# out_it_cov_text.to_csv("../dataKA/latest/it_covidtext.csv", encoding = "utf-8")

In [8]:
import re
import string
from collections import Counter

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import word_tokenize
from nltk.stem import SnowballStemmer

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV


#### Define pre-processing subfunctions ####

def removeNonAlphaNumChars(tweets):
    tweets = [re.sub(r'\-', r' ', tweet) for tweet in tweets]  # replace - with SPACE
    tweets = [re.sub(r'[^\w\s]', r'', tweet) for tweet in tweets]  #remove other NON alpha numeric, excluding whitespace
    return tweets

def replaceContractions(tweets):
    tweets = [re.sub(r"[’\']", r' ', tweet) for tweet in tweets]  ## !!!! WEIRD APOSTROPHE BEWARE !!!
    return tweets

# FROM DEFFRO: https://github.com/Deffro/text-preprocessing-techniques
def removeUnicode(tweets):
    """ Removes unicode strings like "\u002c" and "x96" """
    tweets = [re.sub(r'(\\u[0-9A-Fa-f]+)',r'', tweet) for tweet in tweets]
    tweets = [re.sub(r'[^\x00-\x7f]',r'',tweet) for tweet in tweets]
    return tweets

def replaceURL(tweets):
    """ Replaces url address with "url" """
    tweets = [re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', tweet) for tweet in tweets]
    tweets = [re.sub(r'#([^\s]+)', r'\1', tweet) for tweet in tweets]
    return tweets

def replaceAtUser(tweets):
    """ Replaces "@user" with "atUser" """
    tweets = [re.sub('@[^\s]+','atUser',tweet) for tweet in tweets]
    return tweets

def removeHashtagInFrontOfWord(tweets):
    """ Removes hastag in front of a word """
    tweets = [re.sub(r'#([^\s]+)', r'\1', tweet) for tweet in tweets]
    return tweets
# end DEFFRO


#### DEFINE MAIN PREPROCESSING FUNCTIONS ####

def cleanTweets(tweets):
    tweets = [tweet.lower() for tweet in tweets]  #convert to lowercase
    tweets = replaceURL(tweets) #replace URLs
    tweets = replaceAtUser(tweets)  # replace @user
    tweets = removeHashtagInFrontOfWord(tweets)  # remove hashtag
    tweets = replaceContractions(tweets) # replace ' contractions with space (e.g. l'ordre = l ordre)
    tweets = removeNonAlphaNumChars(tweets)  # remove non alphanumeric characters
    return tweets

def tokenizeTweets(tweets):
    tokens = [nltk.word_tokenize(tweet) for tweet in tweets]
    return tokens

def applyStoplist(tokens):
    tokens = [[word for word in token if len(word) > 2 and (word not in stoplist)] for token in tokens]
    return tokens

def stemTokens(tokens):
    tokens = [[stemmer.stem(word) for word in token] for token in tokens]
    return tokens


In [10]:
# Import preclassified IT tweets (football)
it_preClass = pd.read_csv("C:/Users/pang/Desktop/Stuff/_PhD/PhD Courses/_CSM/Project/preClassified/IT_3cat_worldcup.csv")
# it_preClass = shuffle(it_preClass)

neutral = it_preClass[it_preClass["sentiment"]=="NEUTRAL"]
positive = it_preClass[it_preClass["sentiment"]=="POSITIVE"]
negative = it_preClass[it_preClass["sentiment"]=="NEGATIVE"]

neutral = shuffle(neutral)[:20000]
# positive = shuffle(positive)[:20000]
# negative = shuffle(negative)[:20000]

print("No. neutral: ", len(neutral))
print("No. positive: ", len(positive))
print("No. negative: ", len(negative))

No. neutral:  20000
No. positive:  23552
No. negative:  6542


In [11]:
## Start tweet preprocessing ##
stoplist = stopwords.words('italian')
other_stopwords = "atUser URL"
stoplist = stoplist + other_stopwords.split() # final list of stopwords
# lemmatizer = WordNetLemmatizer() # only for EN
stemmer = SnowballStemmer('italian') # set stemmer

it_preClass_pos_clean = cleanTweets(positive["tweet_text"])
it_preClass_pos_tokens = tokenizeTweets(it_preClass_pos_clean)
it_preClass_pos_tokens = applyStoplist(it_preClass_pos_tokens)
it_preClass_pos_tokens = stemTokens(it_preClass_pos_tokens)

it_preClass_neg_clean = cleanTweets(negative["tweet_text"])
it_preClass_neg_tokens = tokenizeTweets(it_preClass_neg_clean)
it_preClass_neg_tokens = applyStoplist(it_preClass_neg_tokens)
it_preClass_neg_tokens = stemTokens(it_preClass_neg_tokens)

it_preClass_neu_clean = cleanTweets(neutral["tweet_text"])
it_preClass_neu_tokens = tokenizeTweets(it_preClass_neu_clean)
it_preClass_neu_tokens = applyStoplist(it_preClass_neu_tokens)
it_preClass_neu_tokens = stemTokens(it_preClass_neu_tokens)

In [12]:
it_pos_labeled = [[token, "Positive"] for token in it_preClass_pos_tokens]
it_neg_labeled = [[token, "Negative"] for token in it_preClass_neg_tokens]
it_neu_labeled = [[token, "Neutral"] for token in it_preClass_neu_tokens]

it_dataset = it_pos_labeled + it_neg_labeled + it_neu_labeled
it_dataset = shuffle(pd.DataFrame(it_dataset))

# split features and labels for SKLEARN classifier
features = it_dataset.loc[:, 0]
labels  = it_dataset.loc[:, 1]

## Use in-domain test set:

In [13]:
## Prepare for classifier training (SKLEARN) ##

def dummy(doc):
    return doc

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=dummy,preprocessor=dummy,max_df=0.9, min_df=10)
X=cv.fit_transform(features).toarray()
y=labels

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [14]:
## NAIVE BAYES USING SKLEARN ##

#Import Multinomial Naive Bayes model (https://towardsdatascience.com/sentiment-analysis-of-tweets-using-multinomial-naive-bayes-1009ed24276b)
from sklearn.naive_bayes import MultinomialNB

#Create a multinomial NB Classifier
mnb = MultinomialNB()

#Train the model using the training sets
mnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred_NB = mnb.predict(X_test)

In [17]:
# ## RANDOM FOREST USING SKLEARN ##

# # Fitting classifier to the Training set
# from sklearn.ensemble import RandomForestClassifier

# RFclassifier=RandomForestClassifier(n_estimators=200, random_state=0)
# RFclassifier.fit(X_train,y_train)
# # Predicting the Test set results
# y_pred_RF = RFclassifier.predict(X_test)

In [15]:
from sklearn import metrics
from sklearn.metrics import classification_report

print (classification_report(y_test, y_pred_NB))
print("\nAccuracy (NB):", metrics.accuracy_score(y_test, y_pred_NB))

# print ("\n", classification_report(y_test, y_pred_RF))
# print("\nAccuracy (RF):", metrics.accuracy_score(y_test, y_pred_RF))

              precision    recall  f1-score   support

    Negative       0.60      0.60      0.60      1983
     Neutral       0.66      0.59      0.62      5951
    Positive       0.73      0.79      0.76      7095

    accuracy                           0.69     15029
   macro avg       0.66      0.66      0.66     15029
weighted avg       0.69      0.69      0.69     15029


Accuracy (NB): 0.6888681881695389


## Use collected tweets as test set instead of in-domain tweets:

In [13]:
# ### load manually classified tweets (CH/Covid) ###

# it_labeled_all = pd.read_csv("../dataKA/frlabeledAll1.csv")
# it_labeled_cov = pd.read_csv("../dataKA/frlabeledCovid1.csv")

# ## ALL ##
# neu = it_labeled_all[it_labeled_all["label"]=="neu"]
# pos = it_labeled_all[it_labeled_all["label"]=="pos"]
# neg = it_labeled_all[it_labeled_all["label"]=="neg"]
# print("No. neutral (all): ", len(neu))
# print("No. positive (all): ", len(pos))
# print("No. negative (all): ", len(neg))

# # Start preprocessing #
# it_labeled_all_pos_clean = cleanTweets(pos["text"])
# it_labeled_all_pos_tokens = tokenizeTweets(it_labeled_all_pos_clean)
# it_labeled_all_pos_tokens = applyStoplist(it_labeled_all_pos_tokens)
# it_labeled_all_pos_tokens = stemTokens(it_labeled_all_pos_tokens)

# it_labeled_all_neg_clean = cleanTweets(neg["text"])
# it_labeled_all_neg_tokens = tokenizeTweets(it_labeled_all_neg_clean)
# it_labeled_all_neg_tokens = applyStoplist(it_labeled_all_neg_tokens)
# it_labeled_all_neg_tokens = stemTokens(it_labeled_all_neg_tokens)

# it_labeled_all_neu_clean = cleanTweets(neu["text"])
# it_labeled_all_neu_tokens = tokenizeTweets(it_labeled_all_neu_clean)
# it_labeled_all_neu_tokens = applyStoplist(it_labeled_all_neu_tokens)
# it_labeled_all_neu_tokens = stemTokens(it_labeled_all_neu_tokens)

# it_pos_labeled_all = [[token, "Positive"] for token in it_labeled_all_pos_tokens]
# it_neg_labeled_all = [[token, "Negative"] for token in it_labeled_all_neg_tokens]
# it_neu_labeled_all = [[token, "Neutral"] for token in it_labeled_all_neu_tokens]

# it_dataset_all = it_pos_labeled_all + it_neg_labeled_all + it_neu_labeled_all
# it_dataset_all = shuffle(pd.DataFrame(it_dataset_all))


# ## COVID ##
# neu_cov = it_labeled_cov[it_labeled_cov["label"]=="neu"]
# pos_cov = it_labeled_cov[it_labeled_cov["label"]=="pos"]
# neg_cov = it_labeled_cov[it_labeled_cov["label"]=="neg"]
# print("No. neutral (covid): ", len(neu_cov))
# print("No. positive (covid): ", len(pos_cov))
# print("No. negative (covid): ", len(neg_cov))

# ## Start preprocessing ##
# it_labeled_cov_pos_clean = cleanTweets(pos_cov["text"])
# it_labeled_cov_pos_tokens = tokenizeTweets(it_labeled_cov_pos_clean)
# it_labeled_cov_pos_tokens = applyStoplist(it_labeled_cov_pos_tokens)
# it_labeled_cov_pos_tokens = stemTokens(it_labeled_cov_pos_tokens)

# it_labeled_cov_neg_clean = cleanTweets(neg_cov["text"])
# it_labeled_cov_neg_tokens = tokenizeTweets(it_labeled_cov_neg_clean)
# it_labeled_cov_neg_tokens = applyStoplist(it_labeled_cov_neg_tokens)
# it_labeled_cov_neg_tokens = stemTokens(it_labeled_cov_neg_tokens)

# it_labeled_cov_neu_clean = cleanTweets(neu_cov["text"])
# it_labeled_cov_neu_tokens = tokenizeTweets(it_labeled_cov_neu_clean)
# it_labeled_cov_neu_tokens = applyStoplist(it_labeled_cov_neu_tokens)
# it_labeled_cov_neu_tokens = stemTokens(it_labeled_cov_neu_tokens)

# it_pos_labeled_cov = [[token, "Positive"] for token in it_labeled_cov_pos_tokens]
# it_neg_labeled_cov = [[token, "Negative"] for token in it_labeled_cov_neg_tokens]
# it_neu_labeled_cov = [[token, "Neutral"] for token in it_labeled_cov_neu_tokens]

# it_dataset_cov = it_pos_labeled_cov + it_neg_labeled_cov + it_neu_labeled_cov
# it_dataset_cov = shuffle(pd.DataFrame(it_dataset_cov))


# ## split features and labels for SKLEARN classifier
# features_all = fr_dataset_all.loc[:, 0]
# labels_all  = fr_dataset_all.loc[:, 1]
# features_cov = fr_dataset_cov.loc[:, 0]
# labels_cov  = fr_dataset_cov.loc[:, 1]

# features1 = pd.concat([features_all, features], axis=0)
# labels1  = pd.concat([labels_all, labels], axis=0)

# features2 = pd.concat([features_cov, features], axis=0)
# labels2  = pd.concat([labels_cov, labels], axis=0)

# print("No. features (preclassified): ", len(features))
# print("No. features (all): ", len(features1)-len(features))
# print("No. features (covid): ", len(features2)-len(features))

In [ ]:
# ## use manually labelled tweets (ALL) only as test set ##

# X1=cv.fit_transform(features1).toarray()
# y1=labels1
# # X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size = 249, shuffle = "FALSE", stratify=None)

# test_all_index = 500 # USE THIS INSTEAD OF SKLEARN HELPER TO ENSURE CORRECT SPLIT (first 500 are collected tweets)
# X_test1, X_train1 = X1[:test_all_index], X1[test_all_index:]
# y_test1, y_train1 = y1[:test_all_index], y1[test_all_index:]
# print("Size (test set): ", len(X_test1))

# # sklearn multinomial NB classifier #
# mnb.fit(X_train1, y_train1)
# y_pred_NB1 = mnb.predict(X_test1)
# print ("\n", classification_report(y_test1, y_pred_NB1))
# print("\nAccuracy (NB) of manually classified tweets, all:", metrics.accuracy_score(y_test1, y_pred_NB1))

# # sklearn random forest classfier #
# RFclassifier.fit(X_train1,y_train1)
# y_pred_RF1 = RFclassifier.predict(X_test1)
# print ("\n", classification_report(y_test1, y_pred_RF1))
# print("\nAccuracy (RF) of manually classified tweets, all:", metrics.accuracy_score(y_test1, y_pred_RF1))

In [ ]:
# ## use manually labelled tweets (COVID) only as test set ##

# X2=cv.fit_transform(features2).toarray()
# y2=labels2
# # X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size = 250, shuffle = "FALSE", stratify=None)

# test_cov_index = 500 # USE THIS INSTEAD OF SKLEARN HELPER TO ENSURE CORRECT SPLIT
# X_test2, X_train2 = X2[:test_cov_index], X2[test_cov_index:]
# y_test2, y_train2 = y2[:test_cov_index], y2[test_cov_index:]
# print("Size (test set): ", len(X_test2))
# print(y_test2[:3])

# # sklearn multinomial NB classifier #
# mnb.fit(X_train2, y_train2)
# y_pred_NB2 = mnb.predict(X_test2)
# print ("\n", classification_report(y_test2, y_pred_NB2))
# print("\nAccuracy (NB) of manually classified tweets, all:", metrics.accuracy_score(y_test2, y_pred_NB2))

# # sklearn random forest classfier #
# RFclassifier.fit(X_train2, y_train2)
# y_pred_RF2 = RFclassifier.predict(X_test2)
# print ("\n", classification_report(y_test2, y_pred_RF2))
# print("\nAccuracy (RF) of manually classified tweets, all:", metrics.accuracy_score(y_test2, y_pred_RF2))

## Apply trained tool to predict polarity of collected IT tweets

In [16]:
# Using sklearn mNB classifier
labels_all = pd.Series(["tbc"] * len(it_all_tweets))
features_all = cleanTweets(it_all_text)
features_all = tokenizeTweets(features_all)
features_all = applyStoplist(features_all)
features_all = stemTokens(features_all)
features_all = pd.Series(features_all)
labels_train = labels
features_train = features

X_all = cv.fit_transform(pd.concat([features_train, features_all], axis=0)).toarray()
y_all = pd.concat([labels_train, labels_train], axis=0)

cv.fit_transform(features).toarray()
train_index = len(features_train) # USE THIS INSTEAD OF SKLEARN HELPER TO ENSURE CORRECT SPLIT
X_all_train, X_all_test = X_all[:train_index], X_all[train_index:]
y_all_train, y_all_test = y_all[:train_index], y_all[train_index:]

print("Size (test set): ", len(X_all_test))

Size (test set):  43481


In [17]:
# Naive Bayes
mnb.fit(X_all_train, y_all_train)
y_all_pred_NB = mnb.predict(X_all_test)
it_res_all_NB = pd.concat([pd.Series(it_all_tid), pd.Series(it_all_date),
                     pd.Series(it_all_text), pd.Series(y_all_pred_NB)], 
                    axis = 1) 
it_res_all_NB.to_csv("../dataKA/latest/it_res_all_nb.csv", encoding = "utf-8")
print("No. predictions: ", len(y_all_pred_NB))
print(y_all_pred_NB[:10])
print(it_res_all_NB[:5])

No. predictions:  43481
['Positive' 'Positive' 'Neutral' 'Positive' 'Neutral' 'Negative'
 'Negative' 'Neutral' 'Neutral' 'Positive']
                     0                               1  \
0  1244239906630774791  Sun Mar 29 12:28:05 +0000 2020   
1  1244240206485741568  Sun Mar 29 12:29:16 +0000 2020   
2  1244240370415931392  Sun Mar 29 12:29:55 +0000 2020   
3  1244240702164340736  Sun Mar 29 12:31:14 +0000 2020   
4  1244241096156332038  Sun Mar 29 12:32:48 +0000 2020   

                                                   2         3  
0                             @LordOfVenice Forza...  Positive  
1                    @Crocket81893187 :-( \nForza...  Positive  
2  Tra i pochi campionati ancora attivi, quello d...   Neutral  
3  Desideriamo offire il nostro contributo concre...  Positive  
4  Aspettando che tutto torni come prima anche se...   Neutral  


In [ ]:
# # Random forest
# from sklearn.ensemble import RandomForestClassifier
# RFclassifier=RandomForestClassifier(n_estimators=200, random_state=0)

# RFclassifier.fit(X_all_train, y_all_train)
# y_all_pred_RF2 = RFclassifier.predict(X_all_test)
# res_all_RF = pd.concat([pd.Series(fr_all_tweet_id), pd.Series(fr_all_date), 
#                      pd.Series(fr_all_coord), pd.Series(fr_all_place), 
#                      pd.Series(fr_all_text), pd.Series(y_all_pred_RF2)], 
#                     axis = 1) 
# res_all_RF.to_csv("../dataKA/res_all_FR_rf.csv", encoding = "utf-8")
# print("No. predictions: ", len(y_all_pred_RF2))
# print(res_all_RF[:5])